#1 使用神经网络构建mnist手写数字程序

In [5]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

batch_size = 100
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.truncated_normal([784, 300], stddev=0.1)) # 不要初始化为0
b1 = tf.Variable(tf.zeros([300]) + 0.1)
L1 = tf.nn.tanh(tf.matmul(x, W1) + b1)
L1_drop = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.Variable(tf.truncated_normal([300, 300], stddev=0.1)) # 不要初始化为0
b2 = tf.Variable(tf.zeros([300]) + 0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop, W2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.Variable(tf.truncated_normal([300, 100], stddev=0.1)) # 不要初始化为0
b3 = tf.Variable(tf.zeros([100]) + 0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop, W3) + b3)
L3_drop = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.Variable(tf.truncated_normal([100, 10], stddev=0.1)) # 不要初始化为0
b4 = tf.Variable(tf.zeros([10]) + 0.1)

prediction = tf.nn.softmax(tf.matmul(L3_drop, W4) + b4)

# loss = tf.reduce_mean(tf.square(y - prediction))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,
                                                              logits=prediction)) #交叉熵明显收敛更快，而且运行速度也快
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
init = tf.global_variables_initializer()
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(31):
        for batch in range(n_batch):
            batch_xs, batchys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x : batch_xs, y : batchys, keep_prob:0.7})

        test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images,
                                                 y:mnist.test.labels,
                                                 keep_prob:0.7})
        train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images,
                                                  y:mnist.train .labels,
                                                  keep_prob:0.7})
        print("Iter" + str(epoch) + ", Testing Accurancy " + str(test_acc) + ", Training Accurancy " + str(train_acc))

    print("Done")

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


KeyboardInterrupt: 

#2 卷积神经网络构建mnist手写数字数据集

In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

batch_size = 100
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1,28,28,1]) # 大小是28*28的图片，通道数是1

'''
生成截断正态分布
[5,5,1,32]， 5*5是采样窗口，1是输入的通道数，32代表输出的通道数，也就是输出32个卷积核
'''
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1))

'''
每一个卷积核一个偏置值，这里有32个卷积核，就有32个偏置值
'''
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) 
'''
strides的0，3位置都是1，默认的，第1个位置是x方向的步长，第2个位置是y方向的步长
2维卷积操作
'''
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME') + b_conv1)
'''
最大池化
ksize的0，3位置都是1，默认的，第1个位置是x方向的大小，第2个位置是y方向的大小
'''
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME') + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

'''
28*28的图片，经过第一次卷积后还是28*28，这是因为卷积用的SAME padding，第一次池化后变为14*14因为池化步长为2
第二次卷积后为14*14，因为输入为14*14，卷积用的SAME padding，第二次池化后变成7*7，因为池化步长为2
经过上面操作后得到了64张7*7的平面
'''
W_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))#上一层有7*7*64=3136个神经元，全连接第一层有共1024个神经元
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))#1024个神经元，所以有1024个偏置

'''
把上一层的输出扁平化处理，也就是转化成1维
'''
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)

'''
第二个连阶层
'''
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
cross_entroy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entroy)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batchys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x : batch_xs, y : batchys, keep_prob:0.7})

        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,
                                                 y:mnist.test.labels,
                                                 keep_prob:0.7})
        print("Iter" + str(epoch) + ", Testing Accurancy " + str(acc) )

    print("Done")


Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Iter0, Testing Accurancy 0.8547


Iter1, Testing Accurancy 0.9615


Iter2, Testing Accurancy 0.9706


...

Iter9, Testing Accurancy 0.9861


Iter10, Testing Accurancy 0.987


Iter11, Testing Accurancy 0.9871


Iter12, Testing Accurancy 0.99


KeyboardInterrupt: 